# Updated Script for Finance Forecast

In [ ]:
#For the first time, Open Anaconda Prompt as an Administrator and Run this code
# It may take time upto 15 min to necessary packages. Allow the prompt everytime it asks for permission 
conda install pandas-gbq -c conda-forge

# Import or install 5 packages - os, ipywidgets, Pandas, Numpy, Datetime

In [ ]:
#After installing pandas-gbq package run this section if following packages are not installed
!pip install pandas
!pip install numpy

In [1]:
#Importing all necessary packages
import os
import pandas as pd
from pandas.io import gbq
import numpy as np
from datetime import datetime
import ipywidgets as widgets

# Get Last Year's same Month's data from GBQ

In [2]:
start_date = input('Enter a date in YYYY-MM-DD format: ')
end_date = input('Enter a date in YYYY-MM-DD format: ')

query = "SELECT LOC.LVL4_SIC_CD AS REF_SIC_CD, COUNT(DISTINCT SHPMT_INSTC_ID) AS SHIPMENT_COUNT, sum(NET_REVENUE) as NET_REVENUE, SUM(WEIGHT) AS WEIGHT,SUM(NET_REVENUE_LESS_FSC) AS NET_REVENUE_LESS_FSC, safe_divide(ROUND(COALESCE(CAST( ( SUM(DISTINCT (CAST(ROUND(COALESCE( ( (WEIGHT/2000) * SIC_MILES  )  ,0) *(1/1000*1.0), 9) AS NUMERIC) +(cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 1, 15)) as int64) as numeric) * 4294967296 +cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 16, 8)) as int64) as numeric)) * 0.000000001 )) -SUM(DISTINCT (cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 1, 15)) as int64) as numeric) * 4294967296 +cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 16, 8)) as int64) as numeric)) * 0.000000001) )  /(1/1000*1.0) AS FLOAT64), 0), 6), ROUND(COALESCE(CAST( ( SUM(DISTINCT (CAST(ROUND(COALESCE(WEIGHT  ,0)*(1/1000*1.0), 9) AS NUMERIC)+ (cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 1, 15)) as int64) as numeric) * 4294967296 +cast(cast(concat('0x', substr(to_hex(md5(CAST(SHPMT_INSTC_ID   AS STRING))), 16, 8)) as int64) as numeric)) * 0.000000001 )) -SUM(DISTINCT (cast(cast(concat('0x', substr(to_hex(md5(CAST(SHPMT_INSTC_ID   AS STRING))), 1, 15)) as int64) as numeric) * 4294967296 +cast(cast(concat('0x', substr(to_hex(md5(CAST(SHPMT_INSTC_ID   AS STRING))), 16, 8)) as int64) as numeric)) * 0.000000001) ) / (1/1000*1.0) AS FLOAT64), 0), 6)) * 2000 AS LENGTH_OF_HAUL FROM `ltl-prd-datahub.LOOKER_TEMPORARY_LAYER.PC_SHIPMENT` SHP left join `ltl-prd-datahub.LOOKER_TEMPORARY_LAYER.PC_CALENDAR` CAL on SHP.PICKUP_DATE = CAL.CLDR_DATE left join `ltl-prd-datahub.ALL_LOOKER_VIEW.LOC_FREIGHT_POSTAL_AREA_RDS_VW` SIC on SIC.GEO_PSTL_CD = SHP.SHPR_ZIP and SIC.ACTV_FLG = 1 left join `ltl-prd-datahub.ALL_LOOKER_VIEW.LOC_OPS_HIERARCHY_FLATTEN_VW` LOC on SIC.REF_SIC_CD = LOC.LVL5_SIC_CD WHERE PICKUP_DATE between '"+start_date+"' and '"+end_date+"' and CAL.REV_DAY <> 0 and IS_REV_SHPMT = 'Y' and LOC.LVL4_SIC_CD is not null group by 1 order by 1"

df_sic = pd.read_gbq(project_id='ltl-prd-data-user', query=query, dialect='standard')

Enter a date in YYYY-MM-DD format: 2021-08-01
Enter a date in YYYY-MM-DD format: 2021-08-31


In [3]:
df_sic.to_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\Aug-22\V1\SIC_AUG_2021.csv')

In [4]:
#Updated df_sic (including missing SIC_TERM_ZONE_CODES values from Sarah's file) (like NMN,'XNP','XSH','UKC' IN jULY 22 )
df_sic=pd.read_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\Aug-22\V1\SIC_AUG_2021.csv')

# Section for new Budget columns - Weight, Base Revenue (Base+Accessorial) and Billed Revenue (Base+Accessorial+Fuel Surcharge) 

In [4]:
df_base_rev = pd.read_excel(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\New Logic\SIC Level Budgets for BI.xlsx',sheet_name='SIC Budgets for Month')

In [5]:
df_base_rev
df_sic.to_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\New Logic\SIC_JULY_2021.csv')

In [7]:
df_base_rev_tr=np.transpose(df_base_rev)
df_base_rev_tr

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Unnamed: 0,NaN,NaN,MEAN WEIGHT Lb (OB + IB) / 2,OUTBOUND REV-BASE,OUTBOUND REV-ACCESSORIAL,OUTBOUND REV-FUEL SURCHARGE,Outbound Revenue,INBOUND REV-BASE-FREIGHT,INBOUND REV-ACCESSORIAL,INBOUND REV-FUEL SURCHARGE,...,OB % of Total,Outbound Base Revenue (w/o FSC),OB % of Total,Outbound Billed Revenue (w FSC),NaN,Weight,NaN,Trued up Base Revenue,Trued up Billed Revenue,NaN
FN_F03,JUL,Plan,1483047799.112723,143666350.968775,9525602.990347,26849224.103312,180041178.062435,148148680.127381,17218033.162909,26573064.394414,...,NaN,306383907.918245,NaN,360082356.12487,NaN,1483047799.112723,NaN,318558667.249413,371980955.747139,371980955.747139
FN_Div_West,JUL,Plan,629669777.311754,62060262.477232,4313312.489572,11635240.860497,78008815.827301,72130080.226602,8675178.449243,12883621.546753,...,0.433271,132747149.933608,0.433283,156017631.654602,NaN,629669777.311754,NaN,138022115.623945,161173094.846553,162540978.031195
FN_Div_East,JUL,Plan,853378021.800969,82578884.020613,4239453.529436,15213983.242815,102032320.792864,76018599.900779,8542854.713665,13689442.847661,...,0.566729,173636675.100099,0.566717,204064641.585729,NaN,853378021.800969,NaN,180536465.447351,210807775.277202,209439891.537828
FN_Dist_Atlanta,JUL,Plan,67620373.430374,6664000.741414,331726.362723,1219474.139628,8215201.243764,5592744.128312,495307.557619,991766.264491,...,0.045666,13991454.208273,0.04563,16430402.487528,NaN,67620373.430374,NaN,14547431.80134,16973330.452497,16758672.205458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FN_USL,JUL,Plan,2506688.285751,175979.860085,33255.601463,39681.180518,248916.642065,347532.013888,57848.337826,65057.83004,...,0.001366,418470.923094,0.001383,497833.284131,NaN,2506688.285751,NaN,435099.677556,514283.740049,539838.688114
FN_USR,JUL,Plan,2230287.489954,148883.260392,23801.765764,36948.869752,209633.895908,343590.602431,88578.064866,65697.580054,...,0.001127,345370.052312,0.001164,419267.791816,NaN,2230287.489954,NaN,359094.00177,433122.121261,461740.451576
FN_UST,JUL,Plan,8872894.228649,713921.59416,128140.455748,118262.498655,960324.548564,1168097.661893,223147.432775,194989.84433,...,0.005497,1684124.099816,0.005334,1920649.097127,NaN,8872894.228649,NaN,1751046.040131,1984115.229896,2064298.383117
FN_UWC,JUL,Plan,200244.204548,3840.142319,215.53286,538.6861,4594.36128,32797.408628,8810.995729,6838.07019,...,0.000026,8111.350359,0.000026,9188.72256,NaN,200244.204548,NaN,8433.670612,9492.355685,15810.426902


In [8]:
#Replace 1st row with header
new_header = df_base_rev_tr.iloc[0]
df_base_rev_tr = df_base_rev_tr[1:] 
df_base_rev_tr.columns = new_header 
df_base_rev_tr

Unnamed: 0,NaN,NaN,MEAN WEIGHT Lb (OB + IB) / 2,OUTBOUND REV-BASE,OUTBOUND REV-ACCESSORIAL,OUTBOUND REV-FUEL SURCHARGE,Outbound Revenue,INBOUND REV-BASE-FREIGHT,INBOUND REV-ACCESSORIAL,INBOUND REV-FUEL SURCHARGE,...,OB % of Total,Outbound Base Revenue (w/o FSC),OB % of Total,Outbound Billed Revenue (w FSC),NaN,Weight,NaN,Trued up Base Revenue,Trued up Billed Revenue,NaN
FN_F03,JUL,Plan,1483047799.112723,143666350.968775,9525602.990347,26849224.103312,180041178.062435,148148680.127381,17218033.162909,26573064.394414,...,NaN,306383907.918245,NaN,360082356.12487,NaN,1483047799.112723,NaN,318558667.249413,371980955.747139,371980955.747139
FN_Div_West,JUL,Plan,629669777.311754,62060262.477232,4313312.489572,11635240.860497,78008815.827301,72130080.226602,8675178.449243,12883621.546753,...,0.433271,132747149.933608,0.433283,156017631.654602,NaN,629669777.311754,NaN,138022115.623945,161173094.846553,162540978.031195
FN_Div_East,JUL,Plan,853378021.800969,82578884.020613,4239453.529436,15213983.242815,102032320.792864,76018599.900779,8542854.713665,13689442.847661,...,0.566729,173636675.100099,0.566717,204064641.585729,NaN,853378021.800969,NaN,180536465.447351,210807775.277202,209439891.537828
FN_Dist_Atlanta,JUL,Plan,67620373.430374,6664000.741414,331726.362723,1219474.139628,8215201.243764,5592744.128312,495307.557619,991766.264491,...,0.045666,13991454.208273,0.04563,16430402.487528,NaN,67620373.430374,NaN,14547431.80134,16973330.452497,16758672.205458
FN_NAA,JUL,Plan,3012562.876432,209796.74962,9362.64043,38676.40597,257835.79602,266126.368315,16623.896717,42900.941954,...,0.001431,438318.7801,0.001432,515671.592041,NaN,3012562.876432,NaN,455736.227688,532711.498901,537313.575613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FN_USL,JUL,Plan,2506688.285751,175979.860085,33255.601463,39681.180518,248916.642065,347532.013888,57848.337826,65057.83004,...,0.001366,418470.923094,0.001383,497833.284131,NaN,2506688.285751,NaN,435099.677556,514283.740049,539838.688114
FN_USR,JUL,Plan,2230287.489954,148883.260392,23801.765764,36948.869752,209633.895908,343590.602431,88578.064866,65697.580054,...,0.001127,345370.052312,0.001164,419267.791816,NaN,2230287.489954,NaN,359094.00177,433122.121261,461740.451576
FN_UST,JUL,Plan,8872894.228649,713921.59416,128140.455748,118262.498655,960324.548564,1168097.661893,223147.432775,194989.84433,...,0.005497,1684124.099816,0.005334,1920649.097127,NaN,8872894.228649,NaN,1751046.040131,1984115.229896,2064298.383117
FN_UWC,JUL,Plan,200244.204548,3840.142319,215.53286,538.6861,4594.36128,32797.408628,8810.995729,6838.07019,...,0.000026,8111.350359,0.000026,9188.72256,NaN,200244.204548,NaN,8433.670612,9492.355685,15810.426902


In [9]:
df_base_rev_tr['OB_BASE_REV_WO_FSC']=(df_base_rev_tr['OUTBOUND REV-BASE']+df_base_rev_tr['OUTBOUND REV-ACCESSORIAL'])*2
df_base_rev_tr['OB_BASE_REV_WO_FSC_TOT_PCT']=df_base_rev_tr['OB_BASE_REV_WO_FSC']/(df_base_rev_tr['OB_BASE_REV_WO_FSC'].max())
df_base_rev_tr['OB_BASE_REV_W_FSC']=df_base_rev_tr['OB_BASE_REV_WO_FSC']+df_base_rev_tr['OUTBOUND REV-FUEL SURCHARGE']*2
df_base_rev_tr['OB_BASE_REV_W_FSC_TOT_PCT']=df_base_rev_tr['OB_BASE_REV_W_FSC']/(df_base_rev_tr['OB_BASE_REV_W_FSC'].max())
df_base_rev_tr['TOT_BASE_REV_WO_FSC']=df_base_rev_tr['OUTBOUND REV-BASE']+df_base_rev_tr['OUTBOUND REV-ACCESSORIAL']+df_base_rev_tr['INBOUND REV-BASE-FREIGHT']+df_base_rev_tr['INBOUND REV-ACCESSORIAL']
df_base_rev_tr['TOT_BASE_REV_W_FSC']=df_base_rev_tr['TOT_BASE_REV_WO_FSC']+df_base_rev_tr['OUTBOUND REV-FUEL SURCHARGE']+df_base_rev_tr['INBOUND REV-FUEL SURCHARGE']
df_base_rev_tr['TRUED_UP_OB_BASE_REV_WO_FSC']=(df_base_rev_tr['TOT_BASE_REV_WO_FSC'].max())*df_base_rev_tr['OB_BASE_REV_WO_FSC_TOT_PCT']
df_base_rev_tr['TRUED_UP_OB_BASE_REV_W_FSC']=(df_base_rev_tr['TOT_BASE_REV_W_FSC'].max())*df_base_rev_tr['OB_BASE_REV_W_FSC_TOT_PCT']
df_base_rev_tr

C:\Users\mratna\AppData\Local\Temp/ipykernel_11680/1952582157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base_rev_tr['OB_BASE_REV_WO_FSC']=(df_base_rev_tr['OUTBOUND REV-BASE']+df_base_rev_tr['OUTBOUND REV-ACCESSORIAL'])*2
C:\Users\mratna\AppData\Local\Temp/ipykernel_11680/1952582157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base_rev_tr['OB_BASE_REV_WO_FSC_TOT_PCT']=df_base_rev_tr['OB_BASE_REV_WO_FSC']/(df_base_rev_tr['OB_BASE_REV_WO_FSC'].max())
C:\Users\mratna\AppData\Local\Temp/ip

Unnamed: 0,NaN,NaN,MEAN WEIGHT Lb (OB + IB) / 2,OUTBOUND REV-BASE,OUTBOUND REV-ACCESSORIAL,OUTBOUND REV-FUEL SURCHARGE,Outbound Revenue,INBOUND REV-BASE-FREIGHT,INBOUND REV-ACCESSORIAL,INBOUND REV-FUEL SURCHARGE,...,Trued up Billed Revenue,NaN,OB_BASE_REV_WO_FSC,OB_BASE_REV_WO_FSC_TOT_PCT,OB_BASE_REV_W_FSC,OB_BASE_REV_W_FSC_TOT_PCT,TOT_BASE_REV_WO_FSC,TOT_BASE_REV_W_FSC,TRUED_UP_OB_BASE_REV_WO_FSC,TRUED_UP_OB_BASE_REV_W_FSC
FN_F03,JUL,Plan,1483047799.112723,143666350.968775,9525602.990347,26849224.103312,180041178.062435,148148680.127381,17218033.162909,26573064.394414,...,371980955.747139,371980955.747139,306383907.918245,1.0,360082356.12487,1.0,318558667.249413,371980955.747139,318558667.249413,371980955.747139
FN_Div_West,JUL,Plan,629669777.311754,62060262.477232,4313312.489572,11635240.860497,78008815.827301,72130080.226602,8675178.449243,12883621.546753,...,161173094.846553,162540978.031195,132747149.933608,0.433271,156017631.654602,0.433283,147178833.64265,171697696.0499,138022115.623945,161173094.846554
FN_Div_East,JUL,Plan,853378021.800969,82578884.020613,4239453.529436,15213983.242815,102032320.792864,76018599.900779,8542854.713665,13689442.847661,...,210807775.277202,209439891.537828,173636675.100099,0.566729,204064641.585729,0.566717,171379792.164494,200283218.25497,180536465.447351,210807775.277202
FN_Dist_Atlanta,JUL,Plan,67620373.430374,6664000.741414,331726.362723,1219474.139628,8215201.243764,5592744.128312,495307.557619,991766.264491,...,16973330.452497,16758672.205458,13991454.208273,0.045666,16430402.487528,0.04563,13083778.790067,15295019.194185,14547431.80134,16973330.452497
FN_NAA,JUL,Plan,3012562.876432,209796.74962,9362.64043,38676.40597,257835.79602,266126.368315,16623.896717,42900.941954,...,532711.498901,537313.575613,438318.7801,0.001431,515671.592041,0.001432,501909.655082,583487.003007,455736.227688,532711.498901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FN_USL,JUL,Plan,2506688.285751,175979.860085,33255.601463,39681.180518,248916.642065,347532.013888,57848.337826,65057.83004,...,514283.740049,539838.688114,418470.923094,0.001366,497833.284131,0.001383,614615.813261,719354.82382,435099.677556,514283.740049
FN_USR,JUL,Plan,2230287.489954,148883.260392,23801.765764,36948.869752,209633.895908,343590.602431,88578.064866,65697.580054,...,433122.121261,461740.451576,345370.052312,0.001127,419267.791816,0.001164,604853.693453,707500.143259,359094.00177,433122.121261
FN_UST,JUL,Plan,8872894.228649,713921.59416,128140.455748,118262.498655,960324.548564,1168097.661893,223147.432775,194989.84433,...,1984115.229896,2064298.383117,1684124.099816,0.005497,1920649.097127,0.005334,2233307.144576,2546559.487562,1751046.040131,1984115.229896
FN_UWC,JUL,Plan,200244.204548,3840.142319,215.53286,538.6861,4594.36128,32797.408628,8810.995729,6838.07019,...,9492.355685,15810.426902,8111.350359,0.000026,9188.72256,0.000026,45664.079537,53040.835827,8433.670612,9492.355685


In [13]:
df_base_rev_tr
df_base_rev_tr.to_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\New Logic\W_REV_FSC.csv')